# Creating Normalized Tables in PostgresSQL

In [1]:
import psycopg2

### Let's create the connection to the db and get a cursor for query.

In [2]:
try:
    conn = psycopg2.connect("host=localhost user=postgres password=uncle1dee")
except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)    


### Create the database

In [3]:

try:
    cur.execute("create database udacity")
except psycopg2.Error as e:
    print(e)

database "udacity" already exists



In [4]:
# Create all Table

try:
    cur.execute("CREATE TABLE IF NOT EXISTS album_library (album_id int, album_name varchar, artist_id int, year int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS artist_library (artist_id int, artist_name varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    
try:
    cur.execute("CREATE TABLE IF NOT EXISTS song_library (song_id int, album_id int, song_name varchar);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

try:
    cur.execute("CREATE TABLE IF NOT EXISTS song_length (song_id int, song_length int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
 
 
# Insert into the tables
    
try:
    cur.execute("INSERT INTO album_library (album_id, album_name, artist_id, year) VALUES (%s, %s, %s, %s)",
                (1, "Rubber Soul", 1, 1965))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
try:
    cur.execute("INSERT INTO album_library (album_id, album_name, artist_id, year) VALUES (%s, %s, %s, %s)",
                (2, "Let It Be", 1, 1970))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO artist_library (artist_id, artist_name) VALUES (%s, %s)",
                (1, "The Beatles"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        


try:
    cur.execute("INSERT INTO song_library (song_id, album_id, song_name) VALUES (%s, %s, %s)",
                (1, 1, "Michelle"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library (song_id, album_id, song_name) VALUES (%s, %s, %s)",
                (2, 1, "Think For Yourself"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library (song_id, album_id, song_name) VALUES (%s, %s, %s)",
                (3, 1, "In my Life"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library (song_id, album_id, song_name) VALUES (%s, %s, %s)",
                (4, 2, "Let It Be"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library (song_id, album_id, song_name) VALUES (%s, %s, %s)",
                (5, 2, "Across the Universe"))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)

        

try:
    cur.execute("INSERT INTO song_length (song_id, song_length) VALUES (%s, %s)",
                (1, 163))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
try:
    cur.execute("INSERT INTO song_length (song_id, song_length) VALUES (%s, %s)",
                (2, 137))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_length (song_id, song_length) VALUES (%s, %s)",
                (3, 145))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_length (song_id, song_length) VALUES (%s, %s)",
                (4, 240))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_length (song_id, song_length) VALUES (%s, %s)",
                (5, 227))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        

# Print all result

print("\nTable: Album Library\n")
try:
    cur.execute("SELECT * FROM album_library")
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    

print("\nTable: Song Library\n")

try:
    cur.execute("SELECT * from song_library")
except psycopg2.Error as e:
    print("Eroor: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
    
    
print("\nTable: Artist Library\n")
try:
    cur.execute("SELECT * FROM artist_library")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    
print("\nTable: Song length\n")
try:
    cur.execute("Select * from song_length")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


Table: Album Library

(1, 'Rubber Soul', 1, 1965)
(2, 'Let It Be', 1, 1970)

Table: Song Library

(1, 1, 'Michelle')
(2, 1, 'Think For Yourself')
(3, 1, 'In my Life')
(4, 2, 'Let It Be')
(5, 2, 'Across the Universe')

Table: Artist Library

(1, 'The Beatles')

Table: Song length

(1, 163)
(2, 137)
(3, 145)
(4, 240)
(5, 227)


### Let's say we need to do query that gives us:
album_id, album_name, artist_id, year, song_id, song_length, song_name, artist_name

#### we will need to perform a 3 way JOIN on the tables we have created.

In [5]:
try:
    cur.execute("SELECT artist_library.artist_id, artist_name, album_library.album_id,\
    album_name, year, song_library.song_id, song_name, song_length \
    FROM ((artist_library JOIN album_library ON \
    artist_library.artist_id = album_library.artist_id) JOIN \
    song_library ON album_library.album_id = song_library.album_id) JOIN\
    song_length ON song_library.song_id = song_length.song_id;")
    
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'The Beatles', 1, 'Rubber Soul', 1965, 1, 'Michelle', 163)
(1, 'The Beatles', 1, 'Rubber Soul', 1965, 2, 'Think For Yourself', 137)
(1, 'The Beatles', 1, 'Rubber Soul', 1965, 3, 'In my Life', 145)
(1, 'The Beatles', 2, 'Let It Be', 1970, 4, 'Let It Be', 240)
(1, 'The Beatles', 2, 'Let It Be', 1970, 5, 'Across the Universe', 227)


##### We had to do 3 JOIN to achieve this but remember that JOINS are slow and if we have heavy workload that required low latency queries we want to reduce the number of JOINS. Let's think about Denormalising our normalised tables.

With denormalisation we want to think about the queries we are runing and how we can reduce the number of JOINS even if that means duplicating data


##### Query 1:
Select artist_name, album_name, year, song_name, song_length FROM (min number of tables)
I want a list of all songs


##### Query 2:
Select album_name, SUM(song_length) FROM (min number of tables) GROUP BY album_name
I want to know the length of each album in seconds
<hr>

##### Query 1:
Select artist_name, album_name, year, song_name, song_length FROM (min number of tables)
TO reduce the number of tables this is more straight forward. Let's first add song_length to the song_library table and artist_name to album_library

In [6]:
# Create the table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS album_library1 (album_id int, album_name varchar, artist_name varchar, year int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")

try:
    cur.execute("CREATE TABLE IF NOT EXISTS song_library1 (song_id int, album_id int, song_name varchar, song_length int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")


# Insert into all tables
try:
    cur.execute("INSERT INTO album_library1 (album_id, album_name, artist_name, year) VALUES (%s, %s, %s, %s)",
                (1, "Rubber Soul", "The Beatles", 1965))
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
try:
    cur.execute("INSERT INTO album_library1 (album_id, album_name, artist_name, year) VALUES (%s, %s, %s, %s)",
                (2, "Let It Be", "The Beatles", 1970))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        


try:
    cur.execute("INSERT INTO song_library1 (song_id, album_id, song_name, song_length) VALUES (%s, %s, %s, %s)",
                (1, 1, "Michelle", 163))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library1 (song_id, album_id, song_name, song_length) VALUES (%s, %s, %s, %s)",
                (2, 1, "Think For Yourself", 137))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library1 (song_id, album_id, song_name, song_length) VALUES (%s, %s, %s, %s)",
                (3, 1, "In my Life", 145))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library1 (song_id, album_id, song_name, song_length) VALUES (%s, %s, %s, %s)",
                (4, 2, "Let It Be",240))
              
except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        
try:
    cur.execute("INSERT INTO song_library1 (song_id, album_id, song_name, song_length) VALUES (%s, %s, %s, %s)",
                (5, 2, "Across the Universe", 227))
except psycopg2.Error  as e:
    print("Error: Inserting rows")
    
# Print the result
print("\nTable: Album Library 1")
try:
    cur.execute("Select * FROM album_library1")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    

print("\nTable: Song Library 1")
try:
    cur.execute("Select * FROM song_library1")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()


Table: Album Library 1
(1, 'Rubber Soul', 'The Beatles', 1965)
(2, 'Let It Be', 'The Beatles', 1970)

Table: Song Library 1
(1, 1, 'Michelle', 163)
(2, 1, 'Think For Yourself', 137)
(3, 1, 'In my Life', 145)
(4, 2, 'Let It Be', 240)
(5, 2, 'Across the Universe', 227)


##### We can now do a simplified query to get the information. Only one JOIN is needed

##### Query 1:
Select artist_name, album_name, year, song_name, song_length FROM (min number of tables)
I want a list of all songs


In [7]:
try:
    cur.execute("SELECT artist_name, album_name, year, song_name, song_length \
    FROM song_library1 JOIN album_library1 ON song_library1.album_id = album_library1.album_id;")
    
except psycopg2.Error as e:
    print("Error: select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('The Beatles', 'Rubber Soul', 1965, 'Michelle', 163)
('The Beatles', 'Rubber Soul', 1965, 'Think For Yourself', 137)
('The Beatles', 'Rubber Soul', 1965, 'In my Life', 145)
('The Beatles', 'Let It Be', 1970, 'Let It Be', 240)
('The Beatles', 'Let It Be', 1970, 'Across the Universe', 227)


#### Query 2:
Select album_name, SUM(song_length) FROM (min number of tables) GROUP BY album_name I want to know the length of each album in seconds

We can do this by using JOIN but we don't want any JOIN so create another table called album_length



In [8]:
# Create the table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS album_length (song_id int, album_name varchar, song_length int);")
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    
# Insert into the table

try:
    cur.execute("INSERT INTO album_length (song_id, album_name, song_length) VALUES (%s, %s, %s)",
                (1, "Rubber Soul", 163))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

try:
    cur.execute("INSERT INTO album_length (song_id, album_name, song_length) VALUES(%s, %s, %s)",
                (1, "Rubber Soul", 137))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

try:
    cur.execute("INSERT INTO album_length (song_id, album_name, song_length) VALUES(%s, %s, %s)",
                (1, "Rubber Soul", 145))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)
                
try:
    cur.execute("INSERT INTO album_length (song_id, album_name, song_length) VALUES(%s, %s, %s)",
                (2, "Let It Be", 240))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)

try: 
    cur.execute("INSERT INTO album_length (song_id, album_name, song_length) VALUES (%s, %s, %s)",
                (2, "Let It Be", 227))
except psycopg2.Error as e:
    print("Error: Inserting rows")
    print(e)
    
try:
    cur.execute("SELECT * from album_length")
except pycopg2.Error as e:
    print("Error: Select *")
    print(e)
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()
    

(1, 'Rubber Soul', 163)
(1, 'Rubber Soul', 137)
(1, 'Rubber Soul', 145)
(2, 'Let It Be', 240)
(2, 'Let It Be', 227)


#### Now let's run the query 2

In [9]:
try:
    cur.execute("Select album_name, SUM(song_length) FROM album_length GROUP BY album_name;")
except psycopg2.Error as e:
    print("Error: select *")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Rubber Soul', 445)
('Let It Be', 467)


#### Let's do a JOIN on this table so we can get all the information we had on the first table 

In [10]:
try:
    cur.execute("SELECT * FROM album_library JOIN song_library on album_library.album_id = song_library.album_id;")
except psycopg2.Error as e:
    print("Error: Select *")
    print(e)
    
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(1, 'Rubber Soul', 1, 1965, 1, 1, 'Michelle')
(1, 'Rubber Soul', 1, 1965, 2, 1, 'Think For Yourself')
(1, 'Rubber Soul', 1, 1965, 3, 1, 'In my Life')
(2, 'Let It Be', 1, 1970, 4, 2, 'Let It Be')
(2, 'Let It Be', 1, 1970, 5, 2, 'Across the Universe')


### Drop all the tables

In [11]:
    
try:
    cur.execute("DROP table song_library")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

try:
    cur.execute("DROP table song_library1")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

try:
    cur.execute("DROP table album_library")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)
    
try:
    cur.execute("DROP table album_library1")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)
    
try:
    cur.execute("DROP table artist_library")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

try:
    cur.execute("DROP table song_length")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)
try:
    cur.execute("DROP table album_length")
except psycopg2.Error as e:
    print("Error: Dropping table")
    print(e)

### And Finally close your cursor and connection.

In [12]:
cur.close()
conn.close()